# Real-World Regression: Predicting Airbnb Prices in Copenhagen
### From Messy Data to Interpretable Models with SHAP



## Introduction

Welcome to this hands-on walkthrough on real-world regression. Building on the foundational concepts of supervised learning, this session dives into a practical, end-to-end data science project. We will navigate the complexities of a messy, real-world dataset to build a predictive model that delivers tangible business value.

Imagine you are a data scientist hired by a Copenhagen-based property investment firm. The firm wants to expand its portfolio of short-term rental properties but needs a data-driven approach to identify undervalued assets and optimize pricing strategies. Your task is to develop a model that can accurately predict the nightly rental price of a Copenhagen Airbnb listing based on its characteristics.

This problem is a classic regression task. We want to learn a mapping function from property features (`X`) to a continuous price (`Y`). This can be formally expressed as:

$$ \text{Price} = f(\text{location, amenities, host\_characteristics}) + \epsilon $$

Where `f` is the unknown pricing function we aim to estimate, and `ε` represents market noise and unobserved factors. Our goal is to find an estimated function, `f̂`, that generalizes well to new, unlisted properties.

**🎯 Learning Objectives:**

By the end of this walkthrough, you will be able to:

1.  **Execute** an end-to-end data science workflow: from data acquisition and cleaning to model deployment and interpretation.
2.  **Implement** robust feature engineering techniques for messy data types (text, geographic, temporal).
3.  **Construct** a `scikit-learn` preprocessing pipeline to prevent data leakage and streamline modeling.
4.  **Compare** the performance of various regression models, from simple linear baselines to powerful tree-based ensembles.
5.  **Tune** model hyperparameters systematically using an optimization framework (`Optuna`).
6.  **Interpret** complex models and explain individual predictions using SHAP (SHapley Additive exPlanations).
7.  **Translate** model insights into actionable business recommendations for a real-world scenario.

---

### 🛠️ Session Setup: Importing Libraries

First, we load the necessary Python libraries. This project requires a broader toolkit than a simple introduction, including libraries for geographic calculations (`geopy`, `folium`), advanced modeling (`xgboost`), hyperparameter tuning (`optuna`), and model interpretability (`shap`).

In [ ]:
# Core data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re

# Scikit-learn for preprocessing, modeling, and evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

# Advanced libraries
import xgboost as xgb
import shap
import optuna
import folium
from geopy.distance import great_circle

# Settings for reproducibility and display
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
np.random.seed(42)

# Set a professional plot style
sns.set_theme(style="whitegrid", context="talk", palette="viridis")
print("Libraries imported successfully.")

---

## 1. Data Acquisition & Initial Exploration

Our first step is to acquire and understand the raw material for our model: the data. We will use publicly available data from [Inside Airbnb](http://insideairbnb.com/get-the-data/).

### 1.1. The Business Problem

The Copenhagen rental market is dynamic, influenced by high tourism, strict municipal regulations, and distinct neighborhood characteristics (from the historic Indre By to the trendy Vesterbro). Our model must capture these nuances to be effective. The goal is to predict the `price` per night for a given listing.

### 1.2. Data Loading

We'll download the detailed listings data directly into our environment. This ensures our analysis is reproducible.

In [ ]:
# Download the data (this may take a minute)
!wget -q https://data.insideairbnb.com/denmark/hovedstaden/copenhagen/2025-06-27/data/listings.csv.gz -O listings.csv.gz

In [ ]:
# Load the dataset
# We specify low_memory=False to avoid DtypeWarning due to mixed types in some columns.
df_raw = pd.read_csv('listings.csv.gz', compression='gzip', low_memory=False)

In [ ]:
# Get a first impression of the data
print(f"Dataset shape: {df_raw.shape}")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1e6:.2f} MB")
df_raw.head(3)

### 1.3. Initial Data Quality Assessment

Real-world data is rarely clean. A quick look at the column types and non-null counts reveals a lot.

In [ ]:
df_raw.info()

This output shows several challenges:
- **Mixed Data Types:** Some columns that should be numeric are `object` type (e.g., `price`, `host_response_rate`).
- **Widespread Missing Data:** Many columns like `bathrooms`, `bedrooms`, `review_scores_rating`, and most `host_*` columns are missing significant amounts of data. `neighbourhood_group_cleansed` and `license` are entirely empty.

A heatmap can visualize the extent of missing data across the entire dataset.

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_raw.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title("Missing Value Heatmap")
plt.show()

💡 **Pro Tip:** Many columns are almost entirely empty or contain descriptive text not immediately useful for modeling. We'll need to be selective about which features to pursue.

---

## 2. Data Cleaning & Feature Engineering

This is the most critical phase. The quality of our features will determine the performance ceiling of our model. We'll create a clean DataFrame `df` to work with.

**🎯 Section Objectives:**

-   Clean and transform the target variable (`price`).
-   Parse and extract numerical data from text columns.
-   Engineer new features from geographic and temporal data.

### 2.1. Target Variable Cleaning (`price`)

Our target variable `price` is an `object` type because it contains currency symbols (`$`) and commas. Although the currency is Danish Krone (DKK), it's formatted with a dollar sign. We must convert this into a clean numeric format.

In [ ]:
# Make a copy to avoid modifying the original raw data
df = df_raw.copy()

# A more robust function to clean the price column
def clean_price(price_series):
    """
    Cleans a pandas Series of price strings.
    - Handles NaN values gracefully.
    - Removes currency symbols and commas.
    - Converts the result to a numeric type, coercing errors.
    """
    # Use the .str accessor which automatically skips NaN values
    # The regex '[$,]' matches either a '$' or a ','
    cleaned_series = price_series.str.replace(r'[$,]', '', regex=True)
    
    # Convert to numeric, turning any values that can't be converted into NaN
    return pd.to_numeric(cleaned_series, errors='coerce')

df['price_dkk'] = clean_price(df['price'])

# Verify the result by checking the dtype and for any remaining NaNs
print("Cleaned price column data type:", df['price_dkk'].dtype)
print(f"Number of null prices after cleaning: {df['price_dkk'].isnull().sum()}")
df[['price', 'price_dkk']].head(10)

#### Outlier Analysis

Extreme price values (e.g., placeholder values like 0 or luxury listings costing tens of thousands) can skew our model. We'll use winsorization—capping extreme values at a given percentile—to create a more stable target.

In [ ]:
# Describe the price distribution before handling outliers
print("Price distribution before outlier handling:")
print(df['price_dkk'].describe())

In [ ]:
# Winsorize: Cap prices at the 1st and 99th percentiles
lower_bound = df['price_dkk'].quantile(0.01)
upper_bound = df['price_dkk'].quantile(0.99)
df['price_dkk'] = df['price_dkk'].clip(lower=lower_bound, upper=upper_bound)

In [ ]:
# Plot before and after distributions
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.histplot(clean_price(df_raw['price']), ax=axes[0], bins=50, kde=True).set_title("Original Price Distribution (DKK)")
sns.histplot(df['price_dkk'], ax=axes[1], bins=50, kde=True).set_title("Cleaned Price Distribution (DKK)")
plt.tight_layout()
plt.show()

**Business Interpretation:** By cleaning the `price` data, we've created a more stable target variable. The original data had a maximum price over 70,000 DKK, which was likely an outlier. Our cleaned distribution is more focused on the core market, allowing our model to learn more generalizable patterns.

### 2.2. Core Features Processing

We will now process a selection of promising features.

#### Bathroom Text Parsing

The `bathrooms_text` column is descriptive (e.g., "1.5 baths", "Half-bath"). We can extract the numeric value using regular expressions.

In [ ]:
def parse_bathrooms(text_series):
    """
    Parses a text series containing bathroom information into a numeric format.
    Handles values like '1.5 baths', '1 shared bath', and 'Half-bath'.
    """
    # Work with a copy to ensure we are modifying strings
    s = text_series.copy().astype(str).str.lower()

    # Step 1: Replace any string containing 'half-bath' with the numeric string '0.5'.
    # This handles 'Half-bath', 'Private half-bath', 'Shared half-bath', etc.
    s[s.str.contains('half-bath', na=False)] = '0.5'

    # Step 2: Extract the first floating point number found in the remaining strings.
    # The .str.extract() method returns a DataFrame, so we select the first column [0].
    numeric_strings = s.str.extract(r'(\d+\.?\d*)')[0]
    
    # Step 3: Convert the resulting series of strings to numbers.
    # 'errors='coerce'' ensures that any non-numeric results become NaN.
    return pd.to_numeric(numeric_strings, errors='coerce')

df['bathrooms_numeric'] = parse_bathrooms(df['bathrooms_text'])

# Let's run more comprehensive checks to be sure
print("Parsing examples:")
print(f"'1.5 baths' -> {parse_bathrooms(pd.Series(['1.5 baths'])).iloc[0]}")
print(f"'Half-bath' -> {parse_bathrooms(pd.Series(['Half-bath'])).iloc[0]}")
print(f"'2 shared baths' -> {parse_bathrooms(pd.Series(['2 shared baths'])).iloc[0]}")

# The original check should now pass, plus a new one for our special case
assert df[df['bathrooms_text'] == '1.5 baths']['bathrooms_numeric'].iloc[0] == 1.5
assert df[df['bathrooms_text'] == 'Shared half-bath']['bathrooms_numeric'].iloc[0] == 0.5

print("\nAssertions passed successfully!")

#### Boolean Feature Conversion

Columns like `host_is_superhost` and `instant_bookable` are stored as 't'/'f'. We convert them to a binary 0/1 format.

In [ ]:
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})

### 2.3. Geographic Feature Engineering

Location is paramount in real estate. We can create powerful features based on latitude and longitude.

#### Distance to City Center

Let's calculate the distance of each listing to a central landmark, Rådhuspladsen (City Hall Square).

In [ ]:
cph_center = (55.676098, 12.568337) # Rådhuspladsen coordinates

def calculate_distance_to_center(df_geo):
    lat = pd.to_numeric(df_geo['latitude'], errors='coerce')
    lon = pd.to_numeric(df_geo['longitude'], errors='coerce')
    locations = list(zip(lat, lon))
    distances = [
        great_circle(loc, cph_center).kilometers if not (np.isnan(loc[0]) or np.isnan(loc[1])) else np.nan
        for loc in locations
    ]
    return distances

df['distance_to_center_km'] = calculate_distance_to_center(df)

#### Interactive Map of Listings

A `folium` map helps visualize the geographic distribution and price clusters.

In [ ]:
# Create a sample to avoid overloading the map
df_sample = df.sample(n=1000, random_state=42)

# Create a map centered on Copenhagen
map_cph = folium.Map(location=[55.6761, 12.5683], zoom_start=12)

# Add points to the map
for idx, row in df_sample.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.6,
        popup=f"Price: {row['price_dkk']:.0f} DKK"
    ).add_to(map_cph)

# Display map. In some environments (like a standard script),
# you might need to save it to a file: map_cph.save('copenhagen_listings.html')
# or use map_cph.show_in_browser() to open in a new tab.
# map_cph

### 2.4. Temporal Features

How long a listing has been active can be an indicator of its quality and host experience.

In [ ]:
# Convert date columns to datetime objects
df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
df['last_scraped'] = pd.to_datetime(df['last_scraped'], errors='coerce')

# Calculate days since the first review
# We use the latest scrape date as our 'today' for consistency
latest_date = df['last_scraped'].max()
df['days_since_first_review'] = (latest_date - df['first_review']).dt.days

✅ **Checkpoint:** We have now transformed messy, raw data into a structured format with clean, engineered features. Our dataset is much better prepared for modeling.

---

## 3. Train-Test Split Strategy

Before modeling, we must split our data. A simple random split is often sufficient, but for this dataset, we need to be thoughtful.

### 3.1. Preventing Data Leakage

A single host can have multiple listings. If we randomly split the data, we might have listings from the same host in both our training and testing sets. The model could inadvertently learn host-specific pricing patterns, leading to an overly optimistic performance estimate.

⚠️ **Common Pitfall:** Ignoring grouped data structures (like hosts, patients, or stores) during splitting can lead to data leakage and models that fail in production. A more robust method is `GroupKFold`, which ensures all listings from a given host are in the same split. For this workshop, we'll proceed with a standard split but acknowledge this limitation.

We will create three sets: training, validation (for hyperparameter tuning), and testing (for final, unbiased evaluation).

In [ ]:
# Final feature selection for our model
# We select a mix of numeric, categorical, and boolean features.
numeric_features = [
    'accommodates', 'bathrooms_numeric', 'bedrooms', 'beds',
    'review_scores_rating', 'distance_to_center_km', 'days_since_first_review'
]
categorical_features = ['neighbourhood_cleansed', 'room_type', 'property_type']
boolean_features = ['host_is_superhost', 'instant_bookable']

# Combine all features
all_features = numeric_features + categorical_features + boolean_features
target = 'price_dkk'

# Drop rows where our target or key features are missing
df_model = df[all_features + [target]].dropna().copy()

X = df_model[all_features]
y = df_model[target]

# Let's check the dtypes of our final feature set before splitting
X.info()

In [ ]:
# First split: 70% train, 30% temp (for validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Second split: Split the temp set into 50% validation, 50% test (15% of original each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Verify the shapes of our datasets
print(f"Training set shape:   {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape:       {X_test.shape}")

---

## 4. Preprocessing Pipeline

We will use `scikit-learn`'s `Pipeline` and `ColumnTransformer` to create a reproducible preprocessing workflow. This is a cornerstone of production-ready machine learning.

### 4.1. Why Use a Pipeline?

A `scikit-learn` Pipeline is like a recipe for your data. It chains together multiple preprocessing and modeling steps into a single object. This has several major advantages:

1.  **Preventing Data Leakage:** This is the most critical reason. When you perform operations like imputation (filling missing values) or scaling, you must learn the parameters (e.g., the median, the standard deviation) from the **training data only**. A pipeline ensures that when you call `.fit()` on the training data, these parameters are learned and stored. When you call `.transform()` on the validation or test data, it applies the *already learned* parameters, preventing any information from the test set from "leaking" into your training process.
2.  **Reproducibility:** A pipeline encapsulates your entire workflow. Anyone (including your future self) can take your pipeline object and raw data and perfectly reproduce your results.
3.  **Simplicity & Organization:** Instead of manually transforming each dataset (`X_train`, `X_val`, `X_test`), you define the steps once and let the pipeline handle the application, reducing code duplication and the risk of errors.

### 4.2. Scikit-learn Pipeline Architecture

Our pipeline will have separate branches for numeric and categorical features, assembled by a `ColumnTransformer`.

1.  **Numeric Transformer:** Fills missing values (imputes) with the median and then scales the data to have zero mean and unit variance (`StandardScaler`).
2.  **Categorical Transformer:** Fills missing values with a constant ("missing") and then converts categories into numerical format using one-hot encoding. `handle_unknown='ignore'` is crucial for dealing with new categories in the test set that weren't seen during training.
3.  **Boolean Transformer:** Since we already converted these to 0/1, we can simply `'passthrough'` them without any changes.

In [ ]:
# Create the numeric pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create the categorical pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

### 4.3. Assembling the `ColumnTransformer`

This object is the master chef that applies the correct recipe (transformer) to the correct ingredients (columns).

In [ ]:
# The ColumnTransformer applies specified transformers to columns of a DataFrame.
# The structure is a list of tuples: (name, transformer_object, list_of_columns)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', 'passthrough', boolean_features)
    ],
    remainder='drop' # Drop any columns not specified in our feature lists
)

This `preprocessor` is a powerful tool. When we call `.fit(X_train)`, it learns the median, standard deviation, and categories *only from the training data*. When we later call `.transform()`, it applies these learned parameters to any new data we provide.

---

## 5. Model Development

With our data prepared, we can now train and compare several regression models.

### 5.1. Baseline Model

We always start with a simple baseline. A `DummyRegressor` that always predicts the mean of the training target gives us a benchmark. Any real model must perform better than this.

In [ ]:
# Create the full pipeline with the preprocessor and a dummy model
dummy_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', DummyRegressor(strategy='mean'))])

# Train and evaluate the baseline
dummy_pipeline.fit(X_train, y_train)
y_pred_dummy = dummy_pipeline.predict(X_val)
rmse_dummy = np.sqrt(mean_squared_error(y_val, y_pred_dummy))

print(f"Baseline (Mean Predictor) RMSE: {rmse_dummy:,.2f} DKK")

**Business Interpretation:** The baseline model is, on average, wrong by about 739 DKK. This is our starting point. A successful model must significantly reduce this error.

### 5.2. Linear Models

Linear models are fast, interpretable, and provide a great second baseline. We'll try Ridge regression, which adds a penalty for large coefficients to prevent overfitting.

In [ ]:
# Create a Ridge regression pipeline
ridge_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', RidgeCV(alphas=np.logspace(-3, 3, 7)))])

ridge_pipeline.fit(X_train, y_train)
y_pred_ridge = ridge_pipeline.predict(X_val)
rmse_ridge = np.sqrt(mean_squared_error(y_val, y_pred_ridge))

print(f"Ridge Regression RMSE: {rmse_ridge:,.2f} DKK")

### 5.3. Tree-Based Models

Tree-based ensembles like Random Forest and XGBoost are typically more powerful as they can capture non-linear relationships and interactions between features.

In [ ]:
# Random Forest Pipeline
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))

print(f"Random Forest RMSE: {rmse_rf:,.2f} DKK")

In [ ]:
# XGBoost Pipeline
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1))])

xgb_pipeline.fit(X_train, y_train)
y_pred_xgb = xgb_pipeline.predict(X_val)
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))

print(f"XGBoost RMSE: {rmse_xgb:,.2f} DKK")

**Business Interpretation:** Both Random Forest and XGBoost significantly outperform the linear model, reducing the average prediction error to around 500 DKK. This suggests that the relationship between property features and price is non-linear.

### 5.4. Hyperparameter Tuning with Optuna

Our Random Forest model performs well, but we used default settings. We can likely improve it by tuning its hyperparameters using `Optuna`.

🤔 **Discussion Question:** What are hyperparameters, and why is it important to tune them using a separate validation set instead of the test set?

::: {.callout-note collapse="true" title="Click for a possible answer"}
Hyperparameters are settings for a learning algorithm that are not learned from the data itself (e.g., the number of trees in a random forest). They are set by the data scientist before training begins.

It's crucial to use a validation set for tuning to avoid data leakage. If we tune on the test set, we are essentially "peeking" at the final evaluation data and choosing the parameters that work best for it. This leads to an artificially inflated performance score. The test set must be kept completely separate until the very end for a truly unbiased assessment of the final model's generalization ability.
:::

In [ ]:
# Define the objective function for Optuna
# We pass the pre-split data to the function
def objective(trial, X_train, y_train, X_val, y_val):
    # Fit the preprocessor on the training data ONLY
    preprocessor.fit(X_train)
    X_train_transformed = preprocessor.transform(X_train)
    X_val_transformed = preprocessor.transform(X_val)
    
    # Define the search space for hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Create and train the model
    model = RandomForestRegressor(**params)
    model.fit(X_train_transformed, y_train)
    
    # Evaluate on the validation set
    y_pred = model.predict(X_val_transformed)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    return rmse

# Create a study and optimize (run a small number of trials for the workshop)
study = optuna.create_study(direction='minimize')
# Note: For a real project, n_trials would be much larger (e.g., 50-100).
study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=20)

print(f"Best trial RMSE: {study.best_value:.2f} DKK")
print("Best hyperparameters:", study.best_params)

---

## 6. Model Evaluation

Now we select our best model (the tuned Random Forest), train it on the combined training and validation data, and evaluate its final performance on the unseen test set.

### 6.1. Final Model Training and Performance Metrics

In [ ]:
# Get the best hyperparameters from the Optuna study
best_params = study.best_params
best_params['random_state'] = 42
best_params['n_jobs'] = -1

# Create the final, optimized pipeline
final_model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('regressor', RandomForestRegressor(**best_params))])

# Combine train and validation sets for final training
X_train_full = pd.concat([X_train, X_val])
y_train_full = pd.concat([y_train, y_val])

# Train on the full training data
final_model_pipeline.fit(X_train_full, y_train_full)

# Evaluate on the unseen test set
y_pred_final = final_model_pipeline.predict(X_test)

In [ ]:
# Calculate final performance metrics
rmse_final = np.sqrt(mean_squared_error(y_test, y_pred_final))
mape_final = mean_absolute_percentage_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print(f"Final Model Performance on Test Set:")
print(f"RMSE: {rmse_final:,.2f} DKK")
print(f"MAPE: {mape_final:.2%}")
print(f"R² Score: {r2_final:.2f}")

**Business Interpretation:** Our final model has an average error of approximately 480 DKK. The MAPE of ~27% means our predictions are, on average, 27% off from the actual price. The R² score indicates that our model explains about 67% of the variance in Airbnb prices, a respectable result for such a complex market.

### 6.2. Residual Analysis

A residual plot helps diagnose model bias. We look for a random scatter around the zero line.

In [ ]:
residuals = y_test - y_pred_final

plt.figure(figsize=(12, 7))
sns.scatterplot(x=y_pred_final, y=residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residual Plot (Predicted vs. Residuals)")
plt.xlabel("Predicted Price (DKK)")
plt.ylabel("Residuals (Actual - Predicted) (DKK)")
plt.show()

The plot shows some heteroscedasticity (the variance of errors increases as the predicted price increases). Our model is better at predicting prices for cheaper listings than for expensive ones. This is a common finding in price prediction models.

---

## 7. Model Interpretability with SHAP

Our Random Forest model is a "black box." We know it performs well, but we don't know *why*. SHAP (SHapley Additive exPlanations) is a powerful technique to look inside the box.

### 7.1. Global Explanations

First, we need the preprocessed test data and the correct feature names to pass to SHAP. Extracting feature names from a `ColumnTransformer` can be tricky, so we'll break it down.

In [ ]:
# 1. Get the fitted preprocessor from the final pipeline
fitted_preprocessor = final_model_pipeline.named_steps['preprocessor']

# 2. Transform the test data
X_test_transformed = fitted_preprocessor.transform(X_test)

# 3. Extract feature names after one-hot encoding
# Access the 'cat' transformer, then its 'onehot' step, then get the feature names
cat_features_out = fitted_preprocessor.named_transformers_['cat']\
    .named_steps['onehot'].get_feature_names_out(categorical_features)

# 4. Combine all feature names in the correct order
feature_names = numeric_features + list(cat_features_out) + boolean_features

In [ ]:
# Create a SHAP explainer for tree-based models
explainer = shap.TreeExplainer(final_model_pipeline.named_steps['regressor'])
shap_values = explainer.shap_values(X_test_transformed)

A summary plot shows the most important features and their impact. Each dot is a single prediction from the test set.

In [ ]:
# Create the summary plot
shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, max_display=15)

### How to Read This Plot

*   **Features (Y-axis):** The features are listed on the left, ranked by their overall importance to the model's predictions, from most important (`bedrooms`) at the top to least important at the bottom.
*   **Impact on Prediction (X-axis):** The horizontal axis shows the SHAP value. A positive value means that feature pushed the price prediction *higher*, while a negative value pushed it *lower*.
*   **Feature Value (Color):** The color of each dot represents the value of that feature for a given listing. **Red dots are high values** (e.g., many bedrooms, far from the city center), and **blue dots are low values** (e.g., few bedrooms, close to the city center).
*   **Each Dot:** Each dot represents a single prediction (a single listing) from your test set.


### 7.2. Local Explanations

SHAP can also explain individual predictions. Let's find an expensive listing and see why the model priced it that way.

In [ ]:
# Find an expensive listing in the test set
expensive_listing_idx = np.argmax(y_test.values)
expensive_listing_transformed = X_test_transformed[expensive_listing_idx]
expensive_listing_original = X_test.iloc[[expensive_listing_idx]]

print(f"Explaining prediction for listing with actual price: {y_test.iloc[expensive_listing_idx]:,.2f} DKK")
print(f"Model prediction: {y_pred_final[expensive_listing_idx]:,.2f} DKK")
expensive_listing_original

In [ ]:
# Create a force plot for this single prediction
# This interactive plot shows features pushing the prediction higher (red) or lower (blue)
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[expensive_listing_idx, :], 
                X_test_transformed[expensive_listing_idx, :], feature_names=feature_names)

**Business Interpretation:** The force plot shows how each feature "pushes" the prediction away from the baseline average price (`explainer.expected_value`). For this expensive listing, features like having 2 bathrooms, accommodating 6 people, and being an "Entire home/apt" are the primary drivers pushing the price up, while its distance from the center slightly pushes it down.

---

## 8. Business Recommendations

Our model is not just a technical artifact; it's a tool for decision-making.

### 8.1. Key Insights

1.  **Core Amenities Drive Value:** The number of bathrooms, bedrooms, and accommodates are the most critical price drivers. Investments in adding or upgrading these facilities are likely to have the highest ROI.
2.  **Location is Non-Negotiable:** Proximity to the city center is a massive price factor. For listings further out, highlighting other features (e.g., space, quiet) is crucial. Neighborhoods like Indre By and Vesterbro command a significant premium.
3.  **Property Type Matters:** "Entire home/apt" listings are priced significantly higher than private or shared rooms. This model can help quantify that premium.

### 8.2. Deployment Considerations

-   **Pricing Tool:** The model can be deployed as an internal tool for portfolio managers to get a baseline price estimate for a potential new property.
-   **Dynamic Pricing:** The model could be enhanced with seasonality data (e.g., holidays, events) and integrated into a dynamic pricing engine.
-   **Monitoring:** The Copenhagen market changes. The model must be retrained periodically (e.g., quarterly) on new data, and its performance must be monitored for drift.

### 8.3. Limitations & Future Work

-   **Amenities:** We only used a few features. A more detailed parsing of the `amenities` column could uncover value drivers like "hot tub" or "balcony".
-   **Image Data:** We ignored property photos, which contain a wealth of information. A computer vision model could be added to analyze image quality and features.
-   **Seasonality:** Our model doesn't capture seasonal price fluctuations. This would require time-series data.

---

### Group Exercise

**Tasks for Breakout Groups:**

1.  **Feature Engineering Challenge:** The `amenities` column is a string representation of a list of features. Brainstorm and implement a strategy to parse this column. Create binary features for at least three amenities you believe would be important (e.g., `has_wifi`, `has_kitchen`, `has_washer`). Add one of these to the model pipeline and see if it improves the RMSE.

2.  **Error Analysis:** Our model performs worse on expensive listings. Filter the test set to only include the top 10% most expensive properties.
    -   Calculate the RMSE for just this segment. How does it compare to the overall RMSE?
    -   Look at the SHAP summary plot for just this segment. Are the key price drivers different for luxury properties?

3.  **Business Strategy Simulation:** Your firm has identified a property in the *Nørrebro* neighborhood. It has 2 bedrooms, 1 bathroom, and accommodates 4 people. Assume it's an "Entire home/apt" of type "Entire rental unit", has a superhost, is instantly bookable, has a review score of 4.8, is 2.5 km from the city center, and has been reviewed for 500 days. Create a DataFrame with this information and use your final trained model (`final_model_pipeline.predict()`) to estimate its nightly price. The seller is asking for a price that assumes a nightly rate of 1500 DKK. Based on your model's prediction, would you advise the firm that this is a potentially undervalued, fairly valued, or overvalued investment? What other information would you want?